In [ ]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [4]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-MyStudy-LCEL-Runnable")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-MyStudy-LCEL-Runnable


In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)
# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model | StrOutputParser()

In [ ]:
for x in chain.stream({"topic": "코딩"}):
    print(x, end="", flush=True)

코딩은 컴퓨터에게 특정 작업을 수행하도록 명령을 내리는 과정입니다. 프로그래밍 언어를 사용하여 알고리즘과 로직을 작성하며, 이를 통해 다양한 소프트웨어와 애플리케이션을 개발할 수 있습니다. 코딩은 문제 해결 능력과 창의성을 바탕으로 효율적이고 기능적인 프로그램을 만드는 핵심 기술입니다.

In [51]:
from langchain_core.runnables import RunnableParallel, RunnablePick, RunnablePassthrough

runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달합니다. 이는 입력된 데이터를 그대로 통과시키는 역할을 합니다.
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용하여, 'mult' 람다 함수를 할당합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가시킵니다.
    extra=RunnablePassthrough.assign(three_operation=lambda x: x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수를 전달합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값에 1을 더합니다.
    modified=lambda x: x["num"] + 1,
)

In [52]:
runnable.invoke({"num": 11, "age": 28})

{'passed': {'num': 11, 'age': 28},
 'extra': {'num': 11, 'age': 28, 'three_operation': 33},
 'modified': 12}

In [ ]:
from langchain_core.runnables import RunnablePick

runnable_chain = (
    {"num": RunnablePick(keys="num")}
    | PromptTemplate.from_template("{num}에 1을 더하면 {{num + 1}}")
    | ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)
    | StrOutputParser()
)

In [65]:
runnable_chain

{
  num: RunnablePick(keys='num')
}
| PromptTemplate(input_variables=['num'], input_types={}, partial_variables={}, template='{num}에 1을 더하면 {{num + 1}}')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x16a7c8150>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x16a760ed0>, root_client=<openai.OpenAI object at 0x16a74a2d0>, root_async_client=<openai.AsyncOpenAI object at 0x16a7cab10>, model_name='gpt-4.1-nano', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [82]:
# 수식을 아래와 같은 방식으로 작성시 문제를 그대로 출력할 수 없습니다.
formula = {"num": lambda x: x["num"]} | PromptTemplate.from_template(
    "{num}에 1을 더하면 {{num + 1}}"
)
print(f'{ formula.invoke({"num": 11, "age": 28})}')

formula = {
    "num": lambda x: x["num"],
    "formula": lambda x: f"{x['num']}+1",
} | PromptTemplate.from_template("{num}에 1을 더하면 {formula}=")
print(f'{ formula.invoke({"num": 11, "age": 28})}')

text='11에 1을 더하면 {num + 1}'
text='11에 1을 더하면 11+1='
